In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score,mean_squared_error
import seaborn as sns

In [2]:
df = pd.read_csv('final_data.csv')
df = df.drop('Unnamed: 0', axis = 1) # remove extra column

In [3]:
response_var = df[['SalePrice']] # separate the y from the features
feats = df.drop('SalePrice', axis = 1) # features

In [48]:
size(feats)

AttributeError: module 'pandas' has no attribute 'size'

In [4]:
np.random.seed(1) # for reprodusability of results
Xtrain, Xtest, ytrain, ytest = train_test_split(feats, response_var)

In [12]:
regr_1 = RandomForestRegressor()
regr_1.fit(Xtrain, ytrain)

<ipython-input-12-3498f94b6ef8>:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  regr_1.fit(Xtrain, ytrain)


RandomForestRegressor(n_estimators=200)

In [56]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'bootstrap': [True],
    'max_depth': [80, 90, 100, 110],
    'max_features': [40,50,60,70, 80,88],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [100, 200, 300, 1000]
}

grid_search = GridSearchCV(estimator = regr_1, param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2)

In [57]:
grid_search.fit(Xtrain, ytrain)
best_grid = grid_search.best_estimator_
best_grid

grid_search.best_params_
#grid_accuracy = evaluate(best_grid, Xtrain, ytrain)
#print('Improvement of {:0.2f}%.'.format( 100 * (grid_accuracy - base_accuracy) / base_accuracy))

Fitting 3 folds for each of 864 candidates, totalling 2592 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    4.5s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:   35.6s
[Parallel(n_jobs=-1)]: Done 349 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done 632 tasks      | elapsed:  3.3min
[Parallel(n_jobs=-1)]: Done 997 tasks      | elapsed:  5.3min
[Parallel(n_jobs=-1)]: Done 1442 tasks      | elapsed:  8.0min
[Parallel(n_jobs=-1)]: Done 1969 tasks      | elapsed: 11.3min
[Parallel(n_jobs=-1)]: Done 2576 tasks      | elapsed: 15.2min
[Parallel(n_jobs=-1)]: Done 2592 out of 2592 | elapsed: 15.4min finished
C:\Users\ykath\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:765: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


{'bootstrap': True,
 'max_depth': 80,
 'max_features': 40,
 'min_samples_leaf': 3,
 'min_samples_split': 8,
 'n_estimators': 1000}

In [58]:
grid_search.best_score_

0.8543797985826975

In [32]:
y_pred2 = grid_search.predict(Xtest)
mse2 = mean_squared_error(ytest,y_pred2)
rmse2 = np.sqrt(mse2)
rmse2

0.18879214491357066

In [13]:
y_pred1 = regr_1.predict(Xtest)
#y_pred1

In [14]:
mse1 = mean_squared_error(ytest,y_pred1)

In [15]:
rmse1 = np.sqrt(mse1)

In [16]:
rmse1

0.1448257042665818

In [18]:
regr_1.score(Xtest, ytest)

0.8748536900082253

In [51]:
from sklearn.model_selection import GridSearchCV
grid_para_tree = [{
    "min_samples_leaf": range(1, 10),
    "min_samples_split": np.linspace(start=2, stop=30, num=15, dtype=int)
}]
regr_1.set_params(random_state=108)
grid_search_tree = GridSearchCV(regr_1, grid_para_tree, cv=5, scoring='accuracy', n_jobs=-1)
%time grid_search_tree.fit(Xtrain, ytrain)

ValueError: continuous is not supported

In [59]:
from sklearn.linear_model import Lasso

lasso = Lasso()
lasso.set_params(normalize=True, max_iter = 10000) 

l_scores_train = []
l_scores_test = []
l_coefs  = []

alphas = np.linspace(0.000001, .001,100)

for alpha in alphas:
    lasso.set_params(alpha=alpha) # try new alpha level
    lasso.fit(Xtrain, ytrain) # fit test data
    l_scores_train.append(lasso.score(Xtrain,ytrain)) # record R2 train
    l_scores_test.append(lasso.score(Xtest,ytest)) # record R2 test
    l_coefs.append(lasso.coef_) # record coef values
    
l_scores_train = pd.Series(lasso_scores_train) 
l_scores_test  = pd.Series(lasso_scores_test)
l_coefs = pd.DataFrame(l_coefs, index = alphas, columns = Xtrain.columns)  
l_coefs.head()

NameError: name 'lasso_scores_train' is not defined